In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [ ]:
# All functions used below


# Fixed Points
Finding the fixed points $\kappa^*$ using different representations:
* basis vectors using SVD
* basis vectors from E-I perspective

Note:
* plot $\kappa_1,\ \kappa_2$ separately
* plot $\sigma_1,\ \sigma_2$ separately, and how they change with parameters $a,\ b$
* if $J_E$ is not large enough, then changing $a,\ b$ can change the number of FPs
* Also check different transfer functions